In [ ]:
import dropbox
import sys
import os

# Replace with your OAuth2 access token
ACCESS_TOKEN = ''

dbx = dropbox.Dropbox(ACCESS_TOKEN)




def upload_file(file_path, destination_path):
    try:
        with open(file_path, 'rb') as file:
            print(f"Uploading {file_path} to Dropbox")
            dbx.files_upload(file.read(), destination_path)
        
        print(f"File uploaded successfully to {destination_path}")
    except Exception as e:
        print(f"Error uploading file: {str(e)}")



file = 'test.txt'
local_path = f'./{file}'
dropbox_path = f'/ML datasets/{file}'

if not os.path.exists(local_path):
    print(f"Error: File {local_path} does not exist.")
    sys.exit(1)

upload_file(local_path, dropbox_path)


In [ ]:
dbx.users_get_current_account()
for entry in dbx.files_list_folder('/ML datasets').entries:
    print(entry.name)

file = 'test.txt'
local_path = './' + file
dropbox_path = '/ML datasets/'+file
print(dropbox_path)